In [23]:
# импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.impute import SimpleImputer

Считайте датасет из файла train.csv (это данные о выживаемости на Титанике)

In [19]:
# считываем данные из csv-файла
data = pd.read_csv('train.csv')

In [20]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
# первичное исследование данных
print("Информация о данных:")
print(data.info())

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [22]:
# проверяем наличие пропусков
print("\nКоличество пропусков по столбцам:")
print(data.isnull().sum())


Количество пропусков по столбцам:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Удаление малозначимых признаков: убрали бесполезные признаки, такие как имена пассажиров, номера билетов и сведения о каютах.

In [24]:
# Первичная очистка данных
data_cleaned = data.copy()

In [25]:
# Удалим неиспользуемые признаки
data_cleaned = data_cleaned.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

Заполнение пропусков: заменили пропущенные значения возраста средним возрастом всех пассажиров.

In [26]:
# Замена пропусков в возрасте средними значениями
age_imputer = SimpleImputer(strategy='mean')
data_cleaned['Age'] = age_imputer.fit_transform(data_cleaned[['Age']]).ravel()

Преобразование категориальных признаков: перевели пол в числовой формат (женщины → 0, мужчины → 1); посадочные порты закодировали методом one-hot encoding.

In [27]:
# Преобразование категориальных признаков в числовые
data_cleaned['Sex'] = data_cleaned['Sex'].map({'female': 0, 'male': 1})

In [28]:
# Применяем one-hot-кодирование для Embarked
embarked_ohe = pd.get_dummies(data_cleaned['Embarked'], prefix='Embarked', drop_first=True)
data_cleaned = pd.concat([data_cleaned, embarked_ohe], axis=1)
data_cleaned = data_cleaned.drop('Embarked', axis=1)

Деление на тренировочную и тестовую выборки: использовали метод разделения на тренировочное и контрольное множества.

In [29]:
# Готовим входные и выходные данные
X = data_cleaned.drop('Survived', axis=1)
y = data_cleaned['Survived']

Разделим данные на тренировочный и тестовый наборы

Здесь параметр random_state=42 гарантирует, что каждый раз, когда запускается программа, разделение данных на тренировочную и тестовую выборки будет происходить одинаково.

In [30]:
# Деление на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Стандартизация: стандартизировали числовые признаки для улучшения производительности алгоритма.

In [31]:
# Нормализуем числовые признаки
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Произведено измерение качества константного предсказания (например, наиболее частотный класс/случайное предсказание)

In [35]:
from sklearn.dummy import DummyClassifier

# Базовая модель (Dummy Classifier)
# Модель, которая всегда предсказывает наиболее часто встречающийся класс
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train_scaled, y_train)

DummyClassifier(strategy='most_frequent')

In [36]:
# Прогноз базовой модели
y_dummy_pred = dummy_model.predict(X_test_scaled)

In [37]:
# Оценка качества базовой модели
print("Baseline Accuracy:", accuracy_score(y_test, y_dummy_pred))
print("Baseline ROC-AUC:", roc_auc_score(y_test, y_dummy_pred))

Baseline Accuracy: 0.5865921787709497
Baseline ROC-AUC: 0.5


Обучение модели: применили логистическую регрессию.

При обучении логистической регрессии был установлен параметр max_iter=1000, что предотвращает проблемы с ошибкой сходимости и делает процесс обучения стабильным.

In [38]:
# Основная модель (логистическая регрессия)
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [33]:
# Прогнозирование на тестовой выборке
y_pred = log_reg.predict(X_test_scaled)

Оценка качества: посмотрели точность (accuracy), ROC-AUC и отчёт классификации.

ROC-AUC: предпочтительнее для данной задачи, так как показывает способность модели различать классы даже при дисбалансе данных.

In [34]:
# Оценка качества модели
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("Классификация отчет:\n", classification_report(y_test, y_pred))

Accuracy: 0.8100558659217877
ROC-AUC: 0.8001930501930502
Классификация отчет:
               precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



Результаты модели:
* Accuracy (точность): 0.81

Это означает, что примерно 81% случаев классификация была правильной. Однако эта метрика чувствительна к балансу классов. Поскольку в нашем датасете доля погибших больше, чем выживших, высокая точность сама по себе не гарантирует хорошего качества.

* ROC-AUC: 0.80

Эта метрика оценивает способность модели различать классы независимо от конкретного порогового значения. Значение 0.80 свидетельствует о хорошем качестве модели. Чем ближе к единице, тем лучше.

* Precision (точность предсказания выживших): 0.79

Показатель того, какой процент пассажиров, отнесенных моделью к категории "выжившие", действительно выжил. Здесь видим неплохой результат (79%), но он ниже точности для класса "погибшие".

* Recall (полнота): 0.74

Этот показатель отражает долю тех, кто действительно выжил, и кого модель распознала как выжившего. Тут немного хуже результат (74%).

* F1-Score: 0.76

Средняя величина между точностью и полнотой, балансирует обе метрики. Здесь имеем хороший результат, близкий к 0.8